In [1]:
from __future__ import print_function
from keras.preprocessing.image import load_img, img_to_array
# from scipy.misc import imsave
import numpy as np
from scipy.optimize import fmin_l_bfgs_b
import time
import argparse

Using TensorFlow backend.
/usr/local/lib/python2.7/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.21.1) or chardet (2.1.1) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
from keras.applications import vgg19
from keras import backend as K

In [3]:
base_image_path = '/home/rohan/Desktop/kalga.jpg'
style_image_path = '/home/rohan/Desktop/style.jpg'
iterations = 10
total_variation_weight = 1.0
content_weight = 0.025
style_weight = 1.0

In [4]:
width, height = load_img(base_image_path).size
img_nrows = 400
img_ncols = int(width * img_nrows / height)

In [5]:
def preprocess_image(image_path):
    img = load_img(base_image_path,target_size=(img_nrows,img_ncols))
    img = img_to_array(img)
    img = np.expand_dims(img,axis=0)
    img = vgg19.preprocess_input(img)
    return img

In [6]:
def deprocess_image(x):
    if K.image_data_format() == 'channels_first':
        x = x.reshape((3,img_nrows,img_ncols))
        x = x.transpose((1,2,0))
    else:
        x = x.reshape((img_nrows,img_ncols,3))
    
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    
#     BGR to RGB
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [7]:
base_image = K.variable(preprocess_image(base_image_path))
style_image = K.variable(preprocess_image(style_image_path))

In [8]:
if K.image_data_format() == 'channels_first':
    combination_image = K.placeholder((1,3,img_nrows,img_ncols))
else:
    combination_image = K.placeholder((1,img_nrows,img_ncols,3))

In [9]:
input_tensor = K.concatenate([base_image,style_image,combination_image],axis=0)

In [16]:
model = vgg19.VGG19(input_tensor = input_tensor,weights = 'imagenet', include_top = False)
print ('Model loaded')

80101376/80134624 [============================>.] - ETA: 0sModel loaded


In [25]:
model.layers

In [33]:
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])

In [34]:
def gram_matrix(x):
    assert K.ndim(x) == 3
    if K.image_data_format() == 'channels_first':
        features = K.batch_flatten(x)
    else:
        features = K.batch_flatten(K.permute_dimensions(x,(2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram

In [1]:
def style_loss(style,combination):
    assert K.ndim(style == 3)
    assert K.ndim(combination == 3)
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_nrow * img_ncols
    return K.sum(K.square(S - C))/(4.*(channels ** 2) * (size ** 2))

In [2]:
def content_loss(base,combination):
    return K.sum(K.square(combination - base))